# TP 2 - Entrenamiento y evaluación de modelos
## Formula 1 World Championship (1950 - 2023) modificado
El dataset que usamos para entrenar y evaluar los distintos modelos es el generado en la etapa del análisis exploratorio de datos con las distintas conclusiones que pudimos sacar.

## Métrica de performance elegida
Para evaluar los distintos modelos con los que vamos a experimentar, decidimos centrarnos en la métrica de __Precision__.
Esto se debe a que para el caso de uso del modelo es más importante asegurarnos de que la mayoría de los casos que etiquetamos como podio, en realidad lo sean, evitando los falsos positivos, es decir, predecir que un corredor acabará en podio pero en realidad eso no sucede.
La otra opción era trabajar con __recall__, pero determinamos que __precision__ es útil cuando el costo de tener falsos positivos es alto y recall es útil cuando el costo de tener falsos negativos es alto.
Por otro lado, descartamos completamente __accuracy__, ya que nuestro conjunto de datos está claramente desbalanceado hacia el lado de que el corredor no acabará en el podio. Por esto, se podría dar el caso de que el modelo prediga siempre que un corredor no acabará en el podio y tendrá una ‘puntería’ de aproximadamente 85%, como se pudo observar en la etapa de análisis exploratorio de datos.

In [ ]:
# Importamos las dependencias que vamos a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import sklearn_pandas
from matplotlib import gridspec
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import learning_curve
import graphviz
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Quitamos el límite de filas que se muestran en los dataframes
pd.set_option('display.max_rows', None)

In [ ]:
# Importamos el dataset modificado en el EDA.
full = pd.read_csv('./f1_race_podiums.csv')
full.head()

### Tratamiento de valores nulos.
Realizamos el tratamiento de valores nulos como fue determinado en el análisis exploratorio de datos. De esta forma, tendremos los datos preparados para el entrenamiento.

__Time:__ a los valores nulos de esta feature le asignamos el valor que más se repite en el set de datos.

__ds_position:__ en este caso, tenemos valores nulos cuando es la primer temporada del corredor, por lo tanto no tenemos el dato de cómo finalizó la temporada anterior. Le asignamos un valor aleatorio de los disponibles en el dataset.

__cs_position:__ aplicamos la misma lógica que en el anterior.

__ds_points y ds_wins:__ los inicializamos en 0 ya que sería la primer carrera de la temporada.

__cs_points y cs_wins:__ aplicamos la misma lógica que en el anterior.

__q1_ms, q2_ms y q3_ms:__ obtenemos el mayor tiempo de vuelta registrado en la primer etapa de clasificación de una carrera, al cual le sumaremos un segundo y se lo asignaremos a los valores nulos de esa carrera para estas tres features.

In [ ]:
full.isnull().sum()

In [ ]:
# Trabajamos los valores nulos detectados en el EDA
# Null times
moda = str(full['time'].mode()[0]) # Moda: 12:00:00
full['time'].fillna(moda, inplace=True)

# Null ds_positions
filtrado = full[(~full.ds_position.isna())]
# Le asignamos una posicion aleatoria en los casos que sea la primer temporada del corredor.
full['ds_position'] = np.where(full.ds_position.isna(), filtrado.sample(1)['ds_position'], full['ds_position'])

#Null cs_positions
# Le asignamos una posicion aleatoria en los casos que sea la primer temporada del constructor.
full['cs_position'] = np.where(full.cs_position.isna(), filtrado.sample(1)['cs_position'], full['cs_position'])

# Null cs/ds_points and cs/ds_wins
# Los nulos que quedan en ds/cs points y wins, ya sean porque es la primer carrera de la temporada o la primer
# carrera del piloto/escuderia, se inicia en 0
full['cs_points'].fillna(0, inplace=True)
full['cs_wins'].fillna(0, inplace=True)
full['ds_points'].fillna(0, inplace=True)
full['ds_wins'].fillna(0, inplace=True)

# Q1, Q2 and Q3 null lap times
# Nos guardamos el mayor tiempo registrado en cada Q1 para luego asignarlo a los valores nulos de Q1, Q2 y Q3 de esa carrera
# sumandole 1 segundo para diferenciarlo del máximo tiempo obtenido.
max_q1_times = full.groupby('date')['q1_ms'].max()
full['q1_ms'] = full['q1_ms'].fillna(full['date'].map(max_q1_times)+1000)
full['q2_ms'] = full['q2_ms'].fillna(full['date'].map(max_q1_times)+1000)
full['q3_ms'] = full['q3_ms'].fillna(full['date'].map(max_q1_times)+1000)

# Detectamos que hay un caso particular de una carrera donde no se tienen los datos de la clasificación
# A estos casos (24), decidimos eliminarlos
full = full.drop(full[(full.q1_ms.isna()) & (full.q2_ms.isna()) & (full.q3_ms.isna())].index)

full.isnull().sum()

In [ ]:
# Sin feature engineering, hacemos la división del dataset para luego evaluar.
# 60% train, 20% test, 20% validation
train_no_fe, not_train_no_fe = train_test_split(full, train_size=0.6, random_state=42)
validation_no_fe, test_no_fe = train_test_split(not_train_no_fe, train_size=0.5, random_state=42)

## Feature Engineering
__podiums_amnt:__ creación de feature que determina la cantidad de podios del corredor a partir de carreras anteriores.

__experience:__ creación de feature que clasifica la experiencia del corredor en posibles valores de ‘low’, ‘intermediate’ y ‘high’ a partir de la cantidad de carreras hasta la fecha. Una vez clasificado, al ser una variable categórica aplicamos la técnica ‘one-hot encoder’.

__day, month, hour y minute:__ features cradas a partir de variables existentes 'time' y 'date'. De la primera realizamos la extracción de la hora y el minuto y en el caso de la fecha extraemos el día y el mes. Esto lo realizamos para poder escalar las variables ya que eran del tipo datetime.

In [ ]:
# Feature engineering
# podiums_amount
# ordenamos los valores del dataset por driverId y fecha. Acumulamos la cantidad de podios hasta el momento de la carrera.
full = full.sort_values(['driverId', 'date'])
full['acum_podiums'] = full.groupby('driverId')['is_podium'].cumsum()
full['podiums_amnt'] = full['acum_podiums'] - full['is_podium']
full = full.drop('acum_podiums', axis=1)

In [ ]:
full[(full.driverId == 1) & (full.year == 2022)] # Consultamos los valores y verificamos si la cantidad es correcta

In [ ]:
# experience
# usamos una columna auxiliar para ir acumulando la cantidad de carreras
full['aux'] = 1

# Obtenemos la moda de la cantidad de carreras por temporada y usamos este dato para determinar la experiencia según
# X cantidad de temporadas en la F1
races = pd.read_csv('./csvs/races.csv')
races.groupby('year')['round'].max().mode()

# Acumulamos las carreras y le asignamos la experiencia según cantidad aproximada de temporadas en F1
full['acum_races'] = full.groupby('driverId')['aux'].cumsum()
full['experience'] = np.where((full['acum_races'] - 1) <= 16, 'low', np.where((full['acum_races'] - 1) <= 64, 'intermediate', 'high'))

# Si el corredor tiene hasta 16 carreras, consideramos que tiene baja experiencia.
# Si el corredor tiene entre 16 y 64 carreras, consideramos que tiene experiencia intermedia.
# Si el corredor tiene más de 64 carreras, consideramos que tiene alta experiencia.

# Verificamos el dato obtenido con un corredor en particular
full[(full.driverId == 1)]

# Quitamos columnas auxiliares
full = full.drop(['aux', 'acum_races'], axis=1)

full.head()

In [ ]:
# Feature engineering de date y time.
# Separamos estas features en más features para luego poder escalarlas
full['date'] = pd.to_datetime(full['date'])
full['day'] = full['date'].dt.day
full['month'] = full['date'].dt.month
full['time'] = pd.to_datetime(full['time'], format= '%H:%M:%S')
full['hour'] = full['time'].dt.hour
full['minute'] = full['time'].dt.minute

In [ ]:
full.head()

In [ ]:
# Ya tenemos todos los datos trabajados y no hay ningún valor nulo. Pasamos al entrenamiento de los modelos.
# Quitamos el driverId del dataset. También los campos date y time que ya fueron explotados en más campos.
full.drop('driverId', axis=1, inplace=True)
full.drop('time', axis=1, inplace=True)
full.drop('date', axis=1, inplace=True)

## Separación del dataset en train, test y validation
Aplicamos la técnica de Hold Out para dividir el dataset. Utilizamos un 60% para train, y lo restante lo dividimos equitativamente entre test y validation.
Hacer esto nos sirve para aplicar las métricas a estos subconjuntos de datos y poder verificar si el modelo está generalizando bien o está incurriendo en overfitting.

In [ ]:
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(full, train_size=0.6, random_state=42)
validation, test = train_test_split(not_train, train_size=0.5, random_state=42)

full.shape, train.shape, validation.shape, test.shape

In [ ]:
train.head()

### Mapper y transformer
Utilizamos el DataFrameMapper para facilitar la tarea de comunicar Pandas con Sklearn. Para cada feature del dataset, indicamos los transformer a aplicar. Como fue comentado en el análisis exploratorio de datos, usaremos Standar Scaler para variables numéricas y One Hot Encoder para las variables categóricas.

In [ ]:
# Definimos el mapper y transformer a aplicar en cada columna
mapper = DataFrameMapper([
    (['grid'], [StandardScaler()]),
    (['q_position'], [StandardScaler()]),
    (['year'], [StandardScaler()]),
    (['ds_points'], [StandardScaler()]),
    (['ds_position'], [StandardScaler()]),
    (['ds_wins'], [StandardScaler()]),
    (['cs_points'], [StandardScaler()]),
    (['cs_position'], [StandardScaler()]),
    (['cs_wins'], [StandardScaler()]),
    (['q1_ms'], [StandardScaler()]),
    (['q2_ms'], [StandardScaler()]),
    (['q3_ms'], [StandardScaler()]),
    (['podiums_amnt'], [StandardScaler()]),
    (['day'], [StandardScaler()]),
    (['month'], [StandardScaler()]),
    (['hour'], [StandardScaler()]),
    (['minute'], [StandardScaler()]),
    (['circuitRef'], [OneHotEncoder()]),
    (['experience'], [OneHotEncoder()])
])

# Trained with train
mapper.fit(train)

In [ ]:
# Verificamos las features finales luego de aplicar StandarScaler y OneHotEncoder
mapper.transformed_names_

In [ ]:
# Mapper para algoritmos que utilizan árboles
dtree_mapper = DataFrameMapper([
    (['grid'], None),
    (['q_position'], None),
    (['year'], None),
    (['ds_points'], None),
    (['ds_position'], None),
    (['ds_wins'], None),
    (['cs_points'], None),
    (['cs_position'], None),
    (['cs_wins'], None),
    (['q1_ms'], None),
    (['q2_ms'], None),
    (['q3_ms'], None),
    (['podiums_amnt'], None),
    (['day'], None),
    (['month'], None),
    (['hour'], None),
    (['minute'], None),
    (['circuitRef'], [OneHotEncoder()]),
    (['experience'], [OneHotEncoder()])
])
dtree_mapper.fit(train)

In [ ]:
# Mapper para los modelos sin FE
mapper_no_fe = DataFrameMapper([
    (['grid'], [StandardScaler()]),
    (['q_position'], [StandardScaler()]),
    (['year'], [StandardScaler()]),
    (['ds_points'], [StandardScaler()]),
    (['ds_position'], [StandardScaler()]),
    (['ds_wins'], [StandardScaler()]),
    (['cs_points'], [StandardScaler()]),
    (['cs_position'], [StandardScaler()]),
    (['cs_wins'], [StandardScaler()]),
    (['q1_ms'], [StandardScaler()]),
    (['q2_ms'], [StandardScaler()]),
    (['q3_ms'], [StandardScaler()]),
    (['circuitRef'], [OneHotEncoder()]),
])

# Trained with train
mapper_no_fe.fit(train_no_fe)

In [ ]:
dtree_mapper_no_fe = DataFrameMapper([
    (['grid'], None),
    (['q_position'], None),
    (['year'], None),
    (['ds_points'], None),
    (['ds_position'], None),
    (['ds_wins'], None),
    (['cs_points'], None),
    (['cs_position'], None),
    (['cs_wins'], None),
    (['q1_ms'], None),
    (['q2_ms'], None),
    (['q3_ms'], None),
    (['circuitRef'], [OneHotEncoder()]),
])
dtree_mapper_no_fe.fit(train_no_fe)

In [ ]:
# Definimos una función para evaluar distintas métricas de los modelos entrenados.
# Gracias fisa.
def evaluate_model(model, set_names=('train', 'validation'), title='', show_cm=False):
    if title:
        display(title)
        
    final_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': [],
        'AUC_ROC':[],
    }
        
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]  # <- hack feo...

        y = set_data.is_podium
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        final_metrics['AUC_ROC'].append(metrics.roc_auc_score(y, y_pred))
        
        if show_cm:
            cm = metrics.confusion_matrix(y, y_pred)
            cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                                     display_labels=['Not podium', 'Podium'])
            cm_plot.plot(cmap="Blues")
        
    display(pd.DataFrame(final_metrics, index=set_names))

## Grid Search
Realizamos una exploración de hiper-parámetros para cada uno de los modelos que seleccionamos para entrenar.
Esta exploración, fue realizada a través de la técnica de búsqueda en grilla, donde le indicamos los parámetros a modificar y los posibles valores que pueden tomar.
Para elegir los parámetros a tunear y los posibles valores que pueden tomar, nos basamos en la documentación de cada uno de estos modelos de sklearn.
Hacer búsqueda de hiper-parámetros es buena práctica para reducir el overfitting en los modelos.

### Grid Search - Logistic Regression
__penalty:__ tipo de regularización, la regularización L1 puede ayudar a reducir la complejidad del modelo. L2 ayuda a evitar el sobreajuste y mejorar el rendimiento del modelo.

__C:__ valor de regularización, valores pequeños pueden ayudar a evitar el sobreajuste, mientras que valores grandes pueden ayudar a mejorar el ajuste del modelo pero aumenta la chance de sobreajuste.

__Solver:__ es el método de solución, el algoritmo que utiliza para el problema de optimización.

__max_iter:__ cantidad de veces que el modelo se entrena en los datos de entrenamiento.

In [ ]:
# Grid Search
"""
# Logistic Regression
lr_model = LogisticRegression(random_state=42)

lr_parameters = {
    'penalty':['l1', 'l2'],
    'C':[0.1,1,10],
    'solver':['liblinear', 'saga', 'lbfgs'],
    'max_iter':[3000, 3500]
}

lr_clf = GridSearchCV(lr_model, lr_parameters, refit=True, verbose=1)

lr_gs_pipeline = Pipeline([
    ('mapper', mapper),
    ('classifier', lr_clf),
])

lr_gs_pipeline.fit(train, train.is_podium)
lr_clf.best_score_, lr_clf.best_params_
"""

In [ ]:
lr_best_params = {'C': 1, 'max_iter': 3500, 'penalty': 'l1', 'solver': 'saga'}

### Grid Search - K Nearest Neighbors
__n_neighbors:__ número de vecinos a evaluar.

__weights:__ especifica cómo se ponderará la contribución de los vecinos cercanos. En uniform todos los vecinos contribuyen por igual, mientras que en distance los vecinos más cercano tienen más influencia.

__leaf_size:__ asignamos distintos valores para ver cuál de estos aumenta la precisión.

__p:__ para utilizar la distancia Manhattan o distancia euclideana. 

In [ ]:
# KNN
"""
knn_model = KNeighborsClassifier()
knn_parameters = {
    'n_neighbors':[3,5,10,15,20],
    'weights': ['uniform', 'distance'],
    'leaf_size':[5,10,20,30,40],
    'p':[1,2],
}

knn_clf = GridSearchCV(knn_model, knn_parameters, refit=True, verbose=1)
knn_gs_pipeline = Pipeline([
    ('mapper', mapper),
    ('classifier', knn_clf),
])

knn_gs_pipeline.fit(train, train.is_podium)
knn_clf.best_score_, knn_clf.best_params_
"""

In [ ]:
knn_best_params = {'leaf_size': 5, 'n_neighbors': 20, 'p': 1, 'weights': 'uniform'}

### Grid Search - Decision Tree
__Criterion:__ especifica la función para medir la calidad de una división.

__max_depth:__ profundidad máxima del árbol.

__min_samples_split:__ especifica el número mínimo de muestras requeridas para dividir un nodo interno.

__min_samples_leaf:__ especifica el número mínimo de muestras requeridas para formar una hoja.

__max_features:__ especifica el número máximo de features que se deben considerar al buscar la mejor división en cada nodo.

In [ ]:
"""
dtree_model = DecisionTreeClassifier(random_state=42)
dtree_parameters = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1, 2],
    'max_features': [None, 'sqrt', 'log2']
}

dtree_clf = GridSearchCV(dtree_model, dtree_parameters, refit=True, verbose=1)
dtree_gs_pipeline = Pipeline([
    ('mapper', dtree_mapper),
    ('classifier', dtree_clf),
])

dtree_gs_pipeline.fit(train, train.is_podium)
dtree_clf.best_score_, dtree_clf.best_params_
"""

In [ ]:
dtree_best_params = {'criterion': 'gini',
                  'max_depth': 3,
                  'max_features': None,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2}

### Grid Search - Random Forest
Realizamos la búsqueda de los mismos parámetros que en árbol de decisión, agregando:
__n_estimators:__ número de árboles.

__bootstrap:__ si se debe o no realizar muestras con reemplazos.

In [ ]:
# Random Forest
"""
rf_model = RandomForestClassifier(random_state=42)
rf_parameters = {
    'n_estimators':[100,150],
    'criterion':['gini', 'entropy', 'log_loss'],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1, 2],
    'max_features': [None, 'sqrt', 'log2'],
    'bootstrap':[True],
}

rf_clf = GridSearchCV(rf_model, rf_parameters, refit=True, verbose=1)
rf_gs_pipeline = Pipeline([
    ('mapper', dtree_mapper),
    ('classifier', rf_clf),
])

rf_gs_pipeline.fit(train, train.is_podium)
rf_clf.best_score_, rf_clf.best_params_
"""

In [ ]:
rf_best_params = {'bootstrap': True,
              'criterion': 'entropy',
              'max_depth': 7,
              'max_features': None,
              'min_samples_leaf': 1,
              'min_samples_split': 2,
              'n_estimators': 100}

### Grid Search - Gradient Boosting
__n_estimators:__ número de árboles que se usarán en el ensamblado.

__learning_rate:__ una tasa de aprendizaje pequeña supone una mejora más lenta pero adaptándose mejor a los datos.

__max_depth:__ profundidad máxima de cada árbol en el ensamblado

__loss:__ especifica la función de pérdida que se utilizara durante el proceso de entrenamiento.

__max_features:__ número máximo de características que se utilizaran para dividir un nodo interno.

__subsample:__ fracción de muestras que se utilizarán para entrenar cada árbol en el ensamblado.

In [ ]:
# Gradient Boosting
"""
gb_model = GradientBoostingClassifier(random_state=42)
gb_parameters = {
    'n_estimators':[50,100,120],
    'learning_rate':[0.1,0.3],
    'max_depth': [3, 5],
    'loss':['log_loss', 'exponential'],
    'max_features': [None, 'sqrt', 'log2'],
    'subsample':[1.0,0.8],
}

gb_clf = GridSearchCV(gb_model, gb_parameters, refit=True, verbose=1)
gb_gs_pipeline = Pipeline([
    ('mapper', dtree_mapper),
    ('classifier', gb_clf),
])
gb_gs_pipeline.fit(train, train.is_podium)
gb_clf.best_score_, gb_clf.best_params_
"""

In [ ]:
gb_best_params = {'learning_rate': 0.1,
                  'loss': 'log_loss',
                  'max_depth': 3,
                  'max_features': None,
                  'n_estimators': 50,
                  'subsample': 0.8}

### Entrenamiento con Feature Engineering

In [ ]:
# Modelos a utilizar con mejores parámetros

# Logistic Regression
lr_best_params_model = LogisticRegression(**lr_best_params, random_state=42)
lr_best_params_pipeline = Pipeline([
    ('mapper', mapper),
    ('classifier', lr_best_params_model),
])

start = time.time()
lr_best_params_pipeline.fit(train, train.is_podium)
stop = time.time()
print(f"LR Training time: {stop - start}s")

In [ ]:
# Guardamos los tiempos y el algoritmo para luego analizarlo
training_times = [['LR', stop-start]]

In [ ]:
# KNN
knn_best_params_model = KNeighborsClassifier(**knn_best_params)
knn_best_params_pipeline = Pipeline([
    ('mapper', mapper),
    ('classifier', knn_best_params_model),
])

start = time.time()
knn_best_params_pipeline.fit(train, train.is_podium)
stop = time.time()
print(f"KNN Training time: {stop - start}s")

In [ ]:
training_times.append(['KNN', stop-start])

In [ ]:
# Decision Tree

dtree_best_params_model = DecisionTreeClassifier(**dtree_best_params, random_state=42)
dtree_best_params_pipeline = Pipeline([
    ('mapper', dtree_mapper),
    ('classifier', dtree_best_params_model),
])

start = time.time()
dtree_best_params_pipeline.fit(train, train.is_podium)
stop = time.time()
print(f"Decision Tree Training time: {stop - start}s")

In [ ]:
training_times.append(['DTree', stop-start])

In [ ]:
# Random Forest
rf_best_params_model = RandomForestClassifier(**rf_best_params, random_state=42)
rf_best_params_pipeline = Pipeline([
    ('mapper', dtree_mapper),
    ('classifier', rf_best_params_model),
])

start = time.time()
rf_best_params_pipeline.fit(train, train.is_podium)
stop = time.time()
print(f"Random Forest Training time: {stop - start}s")

In [ ]:
training_times.append(['RF', stop-start])

In [ ]:
# Gradient Boosting
gb_best_params_model = GradientBoostingClassifier(**gb_best_params, random_state=42)
gb_best_params_pipeline = Pipeline([
    ('mapper', dtree_mapper),
    ('classifier', gb_best_params_model),
])

start = time.time()
gb_best_params_pipeline.fit(train, train.is_podium)
stop = time.time()
print(f"Gradient Boosting Training time: {stop - start}s")

In [ ]:
training_times.append(['GB', stop-start])

### Evaluación de modelos con Feature Engineering

In [ ]:
# Evaluamos los modelos entrenados
evaluate_model(lr_best_params_pipeline, title='Logistic Regression', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(knn_best_params_pipeline, title='KNN', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(dtree_best_params_pipeline, title='Decision Tree', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(rf_best_params_pipeline, title='Random Forest', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(gb_best_params_pipeline, title='Gradient Boosting', set_names=['train', 'test', 'validation'])

### Entrenamiento de modelos con FE y PCA

In [ ]:
# Modelos a utilizar con mejores parámetros con PCA
# Logistic Regression w/ PCA
lr_best_params_model_pca = LogisticRegression(**lr_best_params, random_state=42)
lr_best_params_pipeline_pca = Pipeline([
    ('mapper', mapper),
    ('pca', PCA(n_components=0.95)),
    ('classifier', lr_best_params_model_pca),
])

start = time.time()
lr_best_params_pipeline_pca.fit(train, train.is_podium)
stop = time.time()
print(f"LR Training time w/ PCA: {stop - start}s")

In [ ]:
training_times.append(['LR PCA', stop-start])

In [ ]:
# KNN w/ PCA
knn_best_params_model_pca = KNeighborsClassifier(**knn_best_params)
knn_best_params_pipeline_pca = Pipeline([
    ('mapper', mapper),
    ('pca', PCA(n_components=0.95)),
    ('classifier', knn_best_params_model_pca),
])

start = time.time()
knn_best_params_pipeline_pca.fit(train, train.is_podium)
stop = time.time()
print(f"KNN Training time w/ PCA: {stop - start}s")

In [ ]:
training_times.append(['KNN PCA', stop-start])

In [ ]:
# Decision Tree w/PCA
dtree_best_params_model_pca = DecisionTreeClassifier(**dtree_best_params, random_state=42)
dtree_best_params_pipeline_pca = Pipeline([
    ('mapper', mapper),
    ('pca', PCA(n_components=0.95)),
    ('classifier', dtree_best_params_model_pca),
])

start = time.time()
dtree_best_params_pipeline_pca.fit(train, train.is_podium)
stop = time.time()
print(f"Decision Tree Training time w/PCA: {stop - start}s")

In [ ]:
training_times.append(['DTree PCA', stop-start])

In [ ]:
# Random Forest w/PCA
rf_best_params_model_pca = RandomForestClassifier(**rf_best_params, random_state=42)
rf_best_params_pipeline_pca = Pipeline([
    ('mapper', mapper),
    ('pca', PCA(n_components=0.95)),
    ('classifier', rf_best_params_model_pca),
])

start = time.time()
rf_best_params_pipeline_pca.fit(train, train.is_podium)
stop = time.time()
print(f"Random Forest Training time w/PCA: {stop - start}s")

In [ ]:
training_times.append(['RF PCA', stop-start])

In [ ]:
# Gradient Boosting w/PCA
gb_best_params_model_pca = GradientBoostingClassifier(**gb_best_params, random_state=42)
gb_best_params_pipeline_pca = Pipeline([
    ('mapper', mapper),
    ('pca', PCA(n_components=0.95)),
    ('classifier', gb_best_params_model_pca),
])

start = time.time()
gb_best_params_pipeline_pca.fit(train, train.is_podium)
stop = time.time()
print(f"Gradient Boosting Training time w/PCA: {stop - start}s")

In [ ]:
training_times.append(['GB PCA', stop-start])

### Evaluación de modelos con FE y PCA

In [ ]:
# Evaluamos los modelos entrenados con PCA
evaluate_model(lr_best_params_pipeline_pca, title='Logistic Regression w/ PCA', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(knn_best_params_pipeline_pca, title='KNN w/ PCA', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(dtree_best_params_pipeline, title='Decision Tree w/PCA', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(rf_best_params_pipeline_pca, title='Random Forest', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(gb_best_params_pipeline_pca, title='Gradient Boosting w/PCA', set_names=['train', 'test', 'validation'])

### Entrenamiento de modelos sin Feature Engineering

In [ ]:
# Modelos a utilizar con mejores parámetros

# Logistic Regression
lr_best_params_model_no_fe = LogisticRegression(**lr_best_params, random_state=42)
lr_best_params_pipeline_no_fe = Pipeline([
    ('mapper', mapper_no_fe),
    ('classifier', lr_best_params_model_no_fe),
])

start = time.time()
lr_best_params_pipeline_no_fe.fit(train_no_fe, train_no_fe.is_podium)
stop = time.time()
print(f"LR w/o FE Training time: {stop - start}s")

In [ ]:
# Guardamos los tiempos y el algoritmo para luego analizarlo
training_times.append(['LR w/o FE', stop-start])

In [ ]:
# KNN
knn_best_params_model_no_fe = KNeighborsClassifier(**knn_best_params)
knn_best_params_pipeline_no_fe = Pipeline([
    ('mapper', mapper_no_fe),
    ('classifier', knn_best_params_model_no_fe),
])

start = time.time()
knn_best_params_pipeline_no_fe.fit(train_no_fe, train_no_fe.is_podium)
stop = time.time()
print(f"KNN w/o FE Training time: {stop - start}s")

In [ ]:
training_times.append(['KNN w/o FE', stop-start])

In [ ]:
# Decision Tree

dtree_best_params_model_no_fe = DecisionTreeClassifier(**dtree_best_params, random_state=42)
dtree_best_params_pipeline_no_fe = Pipeline([
    ('mapper', dtree_mapper_no_fe),
    ('classifier', dtree_best_params_model_no_fe),
])

start = time.time()
dtree_best_params_pipeline_no_fe.fit(train_no_fe, train_no_fe.is_podium)
stop = time.time()
print(f"Decision Tree w/o FE Training time: {stop - start}s")

In [ ]:
training_times.append(['DTree w/o FE', stop-start])

In [ ]:
# Random Forest
rf_best_params_model_no_fe = RandomForestClassifier(**rf_best_params, random_state=42)
rf_best_params_pipeline_no_fe = Pipeline([
    ('mapper', dtree_mapper_no_fe),
    ('classifier', rf_best_params_model_no_fe),
])

start = time.time()
rf_best_params_pipeline_no_fe.fit(train_no_fe, train_no_fe.is_podium)
stop = time.time()
print(f"Random Forest w/o FE Training time: {stop - start}s")

In [ ]:
training_times.append(['RF w/o FE', stop-start])

In [ ]:
# Gradient Boosting
gb_best_params_model_no_fe = GradientBoostingClassifier(**gb_best_params, random_state=42)
gb_best_params_pipeline_no_fe = Pipeline([
    ('mapper', dtree_mapper_no_fe),
    ('classifier', gb_best_params_model_no_fe),
])

start = time.time()
gb_best_params_pipeline_no_fe.fit(train_no_fe, train_no_fe.is_podium)
stop = time.time()
print(f"Gradient Boosting w/o FE Training time: {stop - start}s")

In [ ]:
training_times.append(['GB w/o FE', stop-start])

### Evaluación de modelos sin Feature Engineering

In [ ]:
# Evaluamos los modelos entrenados
evaluate_model(lr_best_params_pipeline_no_fe, title='Logistic Regression w/o FE', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(knn_best_params_pipeline_no_fe, title='KNN w/o FE', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(dtree_best_params_pipeline_no_fe, title='Decision Tree w/o FE', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(rf_best_params_pipeline_no_fe, title='Random Forest w/o FE', set_names=['train', 'test', 'validation'])

In [ ]:
evaluate_model(gb_best_params_pipeline_no_fe, title='Gradient Boosting w/o FE', set_names=['train', 'test', 'validation'])

In [ ]:
# Comparación tiempos de entrenamiento
df_training_times = pd.DataFrame(training_times, columns=['Model', 'Seconds'])
df_training_times

In [ ]:
fig = px.bar(df_training_times.sort_values(by=['Seconds']), x='Seconds', y='Model', orientation='h')
fig.show()

In [ ]:
# Modelo elegido --> Decision Tree with Feature Engineering

## Modelo elegido: Decision Tree con Feature Engineering
Para realizar la elección de este modelo tuvimos en cuenta los siguientes criterios:

__Métrica precision:__ de los modelos que fueron entrenando, se encuentra en los que mayor valor de esta métrica obtuvieron. De todas formas, la mayoría de modelos obtiene valores similares. Por lo tanto, para 'desempatar' esta elección tuvimos en cuenta los otros criterios.

__Tiempo de entrenamiento:__ es el segundo modelo que menos tiempo llevó para entrenarse, luego de KNN por una diferencia muy pequeña como se puede observar en el gráfico anterior.

__Interpretabilidad:__ el modelo de árbol de decisión es de 'caja blanca', esto quiere decir que permite inferir conocimiento de qué es lo que está pasando con solo ver el gráfico del árbol generado.

__Curva de aprendizaje:__ a través del gráfico de la curva de aprendizaje que se muestra a continuación, podemos ver que el modelo tiene buena capacidad de generalización ya que a medida que el set de datos de entrenamiento crece, la métrica de precisión de Train y Validation va incrementando de manera similar.

In [ ]:
# Gráfico Learning Curve
train_sizes, train_scores, test_scores = learning_curve(
estimator=dtree_best_params_pipeline,
X=train,
y=train.is_podium,
train_sizes=np.linspace(0.1, 1.0, 50),
cv=10,
n_jobs=-1,
scoring='precision')

train_mean = np.mean(train_scores, axis=1)

test_mean = np.mean(test_scores, axis=1)

plt.plot(train_sizes, train_mean, color='#0000FF', label='Training')
plt.plot(train_sizes, test_mean, color='#FF0000', label='Validation')

plt.title('Learning Curve - Decision Tree')
plt.xlabel('Train size')
plt.ylabel('Precision')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
graph_data = export_graphviz(
    dtree_best_params_model, 
    out_file=None, 
    feature_names=mapper.transformed_names_,  
    class_names=['Not podium', 'Podium'],  
    filled=True, 
    rounded=True,  
    special_characters=True,
)
graph = graphviz.Source(graph_data)
graph

In [ ]:
df_feat_importances = pd.DataFrame({
    'feature':mapper.transformed_names_,
    'importance':dtree_best_params_model.feature_importances_
}).sort_values(by='importance', ascending=True)

fig = px.bar(df_feat_importances[(df_feat_importances.importance != 0)], x='importance', y='feature', orientation='h')
fig.show()

In [ ]:
resultados = []
for i in range(50):
    test_precision, _ = train_test_split(not_train, train_size=0.5)
    
    y = test_precision.is_podium
    y_pred = dtree_best_params_pipeline.predict(test_precision)
    
    resultados.append(metrics.precision_score(y, y_pred))

In [ ]:
acum = 0
for value in resultados:
    acum += value
    
acum/len(resultados)

## Métrica a presentar al cliente
La métrica a presentar al cliente, sería la obtenida a partir de varias predicciones realizadas con el modelo en datos que no fueron utilizados para el entrenamiento, obtener el promedio de la métrica de precision para que sea más representativa y no quedarnos con la mejor obtenida, ya que estaríamos 'engañando' al cliente ya que no sería la más común.
Con el código anterior, sacamos la conclusión de que el modelo tiene una precisión de, aproximadamente, 66%.
Esto quiere decir que, de las veces que el modelo predice que un corredor acabará en el podio, aproximadamente el 66% de las veces será de esa manera.

Aparte de la métrica, al elegir Decision Tree como modelo, se le puede mostrar el gráfico del árbol al cliente para que vea las variables que tienen mayor peso a la hora de determinar si un piloto acabará en podio o no.

##  Diagramas de dispersión donde se visualicen los aciertos y errores del mismo

In [ ]:
# Gráfico de dispersión para ver true positives/negatives y false positives/negatives  
scatter_y_pred = dtree_best_params_pipeline.predict(validation)
scatter_y_pred

scatter_1 = validation[['grid', 'ds_position', 'is_podium']]
scatter_1['prediction'] = scatter_y_pred

scatter_1['result'] = np.where((scatter_1.is_podium == 1) & (scatter_1.prediction == 1), 'true_positive', 
                              np.where((scatter_1.is_podium == 0) & (scatter_1.prediction == 0), 'true_negative',
                                np.where((scatter_1.is_podium == 0) & (scatter_1.prediction == 1), 'false_positive', 'false_negative')))

In [ ]:
px.scatter(scatter_1, x='grid', y='ds_position', color='result',template='plotly_white',
          color_discrete_map={'true_positive': 'green', 'true_negative': 'yellow', 'false_positive': 'red', 'false_negative':'blue'})

### Conclusiones del gráfico de dispersión
Pudimos observar que el modelo elegido (Decision Tree) clasificará siempre como podio los casos en que la posición de salida (grid) esté entre los primeros 4 puestos, cualquier caso fuera de estos los clasifica como que no hará podio.
Está claro que la feature grid es la que mayor peso tiene.
Habría que buscar la forma de mejorar el modelo para darle más importancia a las otras features y no basarse sólamente en la posición de salida, ya que hay muchos casos donde el corredor no sale entre estos puestos mencionados y termina haciendo podio.

Nota: para observar los distintos casos, en la leyenda de los colores se puede filtrar por los que queremos ver, para que no haya superposición de puntos.